---
title: Statistical inference using weights and survey design
subtitle: 3. R and Stata examples
author: Pierre Walthéry
date: today
date-format: MMMM YYYY
mainfont: Arial
sansfont: Arial
embed-resources: true
resource-path: 
       - pics/
jupyter: stata
execute:
  engine: knitr

format:
# pdf:
#     latex-tinytex: true
#     pdf-engine: lualatex
  revealjs:
     institute: UK Data Service
     scrollable: true
     theme: [default, tooling.scss]
     header: Statistical inference using weights and survey design

     header-logo: UKDS_Logos_Col_Grey_300dpi.png
     embed-resources: true
    
execute: 
   echo: true
   eval: true

filters:
  - reveal-header

---

## Plan
 <!-- If rendering of this script returns an error  --> 
 <!-- run quarto add shafayetShafee/reveal-header
 <!-- In the project  directory -->
-   Three sessions:

  1. Survey design: a refresher

  2. Inference in theory and practice 

  **3.  R and Stata examples**

## This session  

1.  R vs Stata Speak

2.  Means, proportions and CI

3.  When survey design variables are not present

##  Data requirements

- An up to date  copy of R (with the `dplyr`,  `haven`,  `survey` and `Hmisc` packages installed) or Stata  
- An active account with UKDS (for downloading the datasets)
- We will  use data from:

  - The  [2017 British Social Attitudes Survey (BSA)](https://beta.ukdataservice.ac.uk/datacatalogue/studies/study?id=8450): 
  
      https://beta.ukdataservice.ac.uk/datacatalogue/studies/study?id=8450 
  - The [April-June 2022 Quarterly Labour Force Survey](https://beta.ukdataservice.ac.uk/datacatalogue/studies/study?id=8999#!/access-data):
  
      https://beta.ukdataservice.ac.uk/datacatalogue/studies/study?id=8999
  


# 1.  R vs Stata speak
## Survey design in R
- The R *Survey* package [@Lumley2023] provides a comprehensive set of functions for computing  point and variance estimates from survey data.  

- Overall logic:
  - Install the `survey` packages and load it into memory
  - Declare the survey design: ie create a `svydesign` object from the  data
  
  ```
         mydata.s<-svydesign(id=~psu_var,
                             strata=~strata_var, 
                             weights=weight_var,
                             mydata)`
```      
  - Compute estimates with survey-specific functions: `svymean(myvar,mydata.s)`, `svytab(myvar,mydata.s)`, etc

## Survey weights in R
- R  does not provide a unified set of functions or syntax  for computing weighted estimates. 
- Implementation of weighting may vary between packages, but algorithms are usually  described in detail in  package documentation. 
- R Base has only one weight-aware function: `weighted.mean()`
- The Hmisc package offers a more comprehensive set of weighted estimation functions:

  - `wtd.mean()`
  - `wtd.var()`
  - `wtd.quantile()`

- Confidence intervals and standard errors still have to be computed manually

## Survey design in Stata

- Stata provides comprehensive support for computing survey design-informed  estimates from survey data
- Implementation logic similar to R:

  - Declare the survey design using svyset
  
    - `svyset id=psu_var [pweights=weight_var],strata(strata_var)`
    
  - Use `svy:` - prefixed commands for estimation:
  
    - `svy:mean myvar`, `svy:tab myvar` etc...

## Survey weights in Stata - 1

- Users may add sampling weights to most Stata estimation commands, or use survey-specific commands. The latter is recommended. 

- Stata distinguishes between four kinds of (dealing with) weights: 

  - frequency weights (`fweight`), 
  - analytical weights (`aweight`), 
  - importance weights (`iweight`) and 
  - probability weights (`pweight`). 

- These mostly differ in the way standard errors are computed 


## Survey weights in Stata -2 

- Survey weights should be treated as *probability weights* or `pw`. 

- Key estimation commands, such as `summarise` or `tab` do not allow using `pw`:  this is to nudge users to rely on the `svy:` commands instead
.
- 'On the fly' weighting (i.e. not using survey design functions) in Stata consists in the weighting variables being specified between square brackets. 

    - `stata_command myvar [pw=weight_var]`

- It is tempting to to specify instead the wrong  kind of weights function  (`fw` or `aw`) if one does not wish to use the survey design functions. You may get the correct point estimates, but your standard errors are likely to be incorrect  Do this at your own risk

# 2.  Means, proportions and CI


## Identifying the survey design 

We first need to find out about the survey design that was used in the 2017 BSA, and the design variables that are made available in the dataset. Such information can usually be found in the documentation that comes together with the data under the `mrdoc/pdf` folder. 

**Question 1**

- What is the design that was used in this survey (i.e. how many stages were there, and what were the units sampled)? - What were the primary sampling units; the strata (if relevant)?


## Finding  the survey design variables 

Now that we are a bit more familiar with the way the survey was designed, we need to try and identify the design 
variables available in the dataset. The information can usually be found in the user manual or the data dictionary available under `mrdoc/ukda_data_dictionaries.zip` The file may need to be decompressed  separately.

**Question 2**

- What survey design variables are available? 
- Are there any that are missing -- if so which ones? 
- What is the name of the weights variables?

## Preparing the data - R  

```{r}
rm(list=ls())
library(dplyr)                                                              ### Data manipulation functions
library(haven)                                                              ### Importing stata/SPSS files
library(Hmisc)                                                              ### Extra statistical functions
library(survey)                                                             ### Survey design functions

setwd("~/OneDrive/trainings/Stirling_Nov24/")                               ### Edit as appropriate
datadir<-"~/OneDrive/data/"                                                 ### Edit as appropriate
bsa17<-read_dta(paste0(datadir,"bsa/UKDA-8450-stata/bsa2017_for_ukda.dta")) ### Importing Stata format dataset
dim(bsa17)
```

- We can now specify the survey design using:

  - `Spoint` as Primary Sampling Unit, 
  - `StratID` as strata, 
  - `WtFactor` as weights. 

##  Specifying the survey design in R

- We create a `svydesign` object, i.e. a survey design informed copy of the data, which will be used for  subsequent estimation.


```{r 5_2}
bsa17.s<-svydesign(ids=~Spoint, 
                   strata=~StratID, 
                   weights=~WtFactor,
                   data=bsa17)        ### Specifying the survey design
class(bsa17.s)                        ### Examining the svydesign object                                        
summary(bsa17.s)                      ### ... And looking at its content
```


## Mean age and its 95% confidence interval
- We can now produce a first set of estimates using the  survey designinformation. We begin with the mean age of respondents in the sample. 

- We will need to use `svymean()`

```{r mean}
svymean(~RAgeE,bsa17.s)
```

 By default  `svymean()` computes the standard error of the mean. We need to  
 embed it within `confint()` in order to get a confidence interval. 


```{r 5_3}
confint(                                  ### Computing the confidence interval...
  svymean(~RAgeE,bsa17.s)                 ### And the mean
  )                   

round(                                    ### Rounding the results to one decimal point
  c(
    svymean(~RAgeE,bsa17.s),              ### Computing the mean...
    confint(svymean(~RAgeE,bsa17.s))      ### And its 95% CI
    ),
  1) 
```

## Question 3
- What would be the consequences of:
  - weighing but not accounting for the sample design; 
  - not using  weights and accounting for the sample design...

- When: 
  - inferring the mean value of the population age?
  - computing the uncertainty  of the  estimate of  population age? 

## Answer - 1

- When using on the spot weighting, we need to compute means and CI separately: 


```{r}
a.m<-wtd.mean(                               ### Weighted mean function from Hmisc
  bsa17$RAgeE,
  bsa17$WtFactor,
  normwt = T)                                ### Option specific to survey weights  

### Computation of the standard error by hand...
a.se<-sqrt(
          wtd.var(bsa17$RAgeE,               ### ... using the weighted variance function from Hmisc
                  bsa17$WtFactor,
                  normwt = T)
          )/
       sqrt(
         nrow(bsa17)                         ### ... shortcut to sample size
         )

c(a.m,                                       ### Concatenating the mean..
  a.m-1.96*a.se,                             ### ... the lowbound of the CI
  a.m+1.96*a.se)                             ### ... and the higher bound
```


 
- Unweighted estimates:


```{r}
ua.m<-mean(bsa17$RAgeE)                     ### mean() function from R Base
  
ua.se<-sd(bsa17$RAgeE)/                     ### ... standard errors
      sqrt(nrow(bsa17))    ##

c(ua.m,                                     ### and putting it all together
  ua.m-1.96*ua.se,
  ua.m+1.96*ua.se
  )

```


## Answer - 2

- Not using weights results in  overestimating the mean age in the population (of those aged 18+) by about 4 years. 
- This is likely to be due to the fact that older respondents are more likely to take part to surveys. 
- Using `on the fly` weights does not alter the value of the estimated population mean when compared with SD informed estimates...
- But would lead us to overestimating the precision/underestimate the uncertainty of our estimate -- by about plus or minus 3 months. 

## Stata version
- Opening the dataset and declaring the survey design

In [ ]:
use ~/OneDrive/data/bsa/UKDA-8450-stata/bsa2017_for_ukda.dta,clear

svyset Spoint [pw=WtFactor], strata(StratID) 

##
- Computing the survey design-informed  version of the mean...


In [ ]:
svy: mean RAgeE

## 
- And the other two: 

In [ ]:
mean RAgeE [pw=WtFactor]
mean RAgeE

## Computing a proportion and its 95% confidence interval
- We can similarly estimate the distribution of a categorical variable in the population by estimating proportions (or percentages)
- Let's look at the proportion of people who declare that they are interested in politics. 
- This is the `Politics` variable in the BSA. 
- It has five categories ranging from 1 'A great deal' to 5- 'Not at all'.
- We could recode 1 and 2 `A great deal` and `quite a lot` into 'Significantly', but since we are only interested in estimating the confidence intervals, we will select the relevant values 'on the go'. 

## Let's explore the variable
- Phrasing of the question:

```{r 5_4_1} 
attr(bsa17$Politics,"label")     
```


- Value labels:

```{r 5_4_2}
attr(bsa17$Politics,"labels")     
```

- Sample distribution


```{r 5_4_3}
table(
  droplevels(
             as_factor(bsa17$Politics)
             )
  ) 
``` 


##
- Neater output

```{r 5_5}
round(
  100*
    prop.table(
      svytable(~(Politics==1 | Politics==2),bsa17.s)
      ),
  1)
```

- Let us now estimate the confidence intervals for  these proportions. Traditional statistical software usually does  this without showing us  what is happening under the bonnet. 
R requires more coding, but also a better understanding of what is actually estimated. 

- Confidence intervals for proportions of categorical variables are in fact  computed as a sequence of binomial/dichotomic estimations -- i.e. one for each category. 

##
- In R this is  specified explicitly via the `svyciprop()` and `I()` functions:

  - The former computes the proportion and its confidence interval (by default 95%)...
  - whereas the latter allows us to define the category of interest.


```{r 5_6_1}
p<-svyciprop(~I(Politics==1 | Politics==2),bsa17.s)
p
```                     


- A neater version:

```{r 5_6_2}
round(100*
        c(p[1],
attr(p,"ci")),
1
)
```


## Question 4
- What is the proportion of respondents aged 17-34 in the sample, as well as its 95% confidence interval? 

  - You can use `RAgecat5`

## Answer
- The proportion of 17-34 year old in the sample is: 


```{r} 
a<-svyciprop(~I(RAgecat5 == 1),bsa17.s)

round(
  100*
      a[1],1)
```

and its 95% confidence interval:                 

```{r} 
round(
    100*
    attr(a,"ci"),
    1)
```


## Stata

* Proportions and Question 4

In [ ]:
recode Politics 1 2 =1 3/8=2,gen(Politics2)
svy:ta Politics2
svy:ta Politics2, percent ci
svy:ta RAgecat5, percent ci

## Computing domain estimates -1 
- Computing estimates for subpopulation adds a layer of complexity to the above example. 
- Weights are usually  designed to make the  full  sample representative of the population
- Computing estimates for part of the sample only, therefore using a fraction of these weights may affect the estimates. 
- It is recommended to use commands that take into account the entire distribution of the weights instead.

In R, the command that does this is `svyby()`

## Computing domain estimates - 2

- Say we would like to compute the mean age of BSA respondents by Government Office Regions, we need to specify:

  - The outcome variable whose estimate we want to compute: i.e. `RAgeE`
  - The grouping variable(s) `GOR_ID`
  - The estimate function we are going to use here: `svymean`, the same as  we used before
  - And the type of type of variance estimation we would like to see displayed i.e. standard errors or confidence interval  

##

```{r 5.7}
d<-      svyby(~RAgeE,
             by=~as_factor(GOR_ID),
             svymean,
             design=bsa17.s,
             vartype = "ci")
round(d[-1],1)
```

  *Note:* we used `[-1]` above  in order to remove a column with alphanumeric values (the region names), so that we could round the results without getting an error.

##
-  Results  suggest that: 

   - the population in  London is among the youngest in the country;
   - those in the South West are among the oldest -- their respective 95% confidence intervals do not overlap. 
   - The same cannot be said of differences between London and the South East, as the CIs partially overlap.

- We can adopt  a similar approach with proportions: we just need to specify the category of the variable we are interested in as an outcome, for instance respondents who are significantly interested in politics, and replace `svymean` by `svyciprop`.

##

```{r 5.8}
c<-round(
      100*
      svyby(~I(Politics==1 | Politics==2),
            by=~as_factor(GOR_ID),
            svyciprop,
            design=bsa17.s,
            vartype = "ci")[-1],
            1)
c
```

## Question 5
- What is the 95% confidence interval for the proportion of people interested in politics in the North East? 
- Is the proportion likely to be different in London? In what way? 
- What is the region of the UK for which the precision of the estimates is likely to be the smallest?

## Answer

- The 95% confidence interval for the proportion of people very interested in politics in the North East is  `r as.numeric(c[1,2:3])`. 
- By contrast, it is  `r as.numeric(c[7,2:3])` in London. 
- The region with the lowest precision of estimates (i.e. the widest confidence interval) is Wales, with more than   20  percentage point difference between the upper and lower bounds of the confidence interval.

## Question 6
- Using interest in politics as before, and three category age `RAgecat5`: 

  - Produce a table showing the proportion of respondents significantly interested in Politics by age group and gender
  - Assess whether the age difference in interest for politics is similar for each gender.
  - Based on the data, is it fair to say that men aged under 35 tend to  be more likely to declare  themselves  interested  in politics  than women aged 55 and above?

## Question 6


```{r 6.1}
round(
      100*
        svyby(~I(Politics==1 | Politics==2),
              by=~as_factor(RAgecat5)+as_factor(Rsex),
              svyciprop,
              design=bsa17.s,
              vartype = "ci")[c(-8,-4),c(-2,-1)],1)
``` 

- Older respondents both male and female tend to be more involved in politics than younger ones.

- The confidence interval for the proportion of men under 35 and women above 55 interested in politics overlaps; it is unlikely that they  differ in the population.

## Stata


# 3 Correcting for the absence of survey design variables 

##  Regional employed population with the End User License LFS
- We are using the Quarterly Labour Force Survey, April-July 2022 issue. 
- As a rule, EUL versions of the LFS do not include survey  design variables. 
- The LFS   come with  two weight variables:

  - `pwt22` for estimation with the whole sample
  - `piwt22` for estimation of earnings using  respondents currently in employment (and accounting for  the high level of non response for the earnings variables) 

- Estimation without  accounting for sample design will likely be biased and should be reported as such including warnings, even if its nature (over or underestimation) and size are not known. 
##
- An alternative is to look for design effects tables published by the data producer which could be used to correct for the bias.

- The Office for National Statistics regularly publishes Deft tables for the LFS, but only  for their headline statistics.  
- Let's first produce uncorrected 'naive' estimates of the regional population.
- We will still use the survey design functions, but define a SRS design 

    

```{r  5.9}
lfs<-read_dta((paste0(datadir,"lfs/UKDA-8999-stata/lfsp_aj22_eul_pwt22.dta")))%>%
     select(PWT22,PIWT22,GOVTOF2,URESMC,ILODEFR)
lfs.s<-svydesign(ids=~1,weights=~PIWT22,data=lfs%>%filter(ILODEFR==1)) 
nv<-round(confint(svytotal(~as_factor(GOVTOF2),lfs.s)))
```


![Test](pics/lfs_vol1_SE.png)

For some reason, the  variable  for Government Office Region  - GOVTOF - used by the ONS differentiates Merseyside, and not the one we have in the data. We therefore need to recode the  original variable accordingly, using another, more detailed variable.
We also have to r

```{r 5.10}
lfs<-lfs%>%mutate(govtof=ifelse(URESMC==15,3,GOVTOF2)) # Identifying Merseyside
lfs$govtof.f<-as.ordered(lfs$govtof)                   # Converting into factor
levels(lfs$govtof.f)<-c(names(attr((lfs$GOVTOF2),"labels")[3:4]),"Merseyside",names(attr((lfs$GOVTOF2),"labels")[5:14])) # Adding factor levels from existing labels
table(lfs$govtof.f)                      
```

Let us now check the results:


```{r 5.12}
lfs.s<-svydesign(ids=~1,weights=~PIWT22,data=lfs%>%filter(ILODEFR==1)) 
round(
  confint(
    svytotal(~govtof.f,lfs.s)
    )
  )
```


We can now import the design factors from the LFS documentation. This has to be done by hand, by  copying the relevant values of the design factors. 



```{r 5.13}
tot<-data.frame(svytotal(~govtof.f,lfs.s))
tot$deft<-c(0.8712,1.0857,1.3655,1.0051,0.9634,
            1.0382,0.8936,1.3272,0.9677,0.9137,
            1.0012,1.0437,0.7113)
tot["2.5%"]<-tot$total-(1.96*tot$SE*tot$deft)
tot["97.5%"]<-tot$total+(1.96*tot$SE*tot$deft)
# Cleaning up the labels
rownames(tot)<-substr(rownames(tot),9,nchar(rownames(tot)))
```



- Any comments: pierre.walthery@manchester.ac.uk